<a href="https://colab.research.google.com/github/ashaaher/Deep-Learning/blob/master/Ungraded-5/1_TensorFlowQuickStart.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Importing TensorFlow

In [0]:
import tensorflow as tf

from tensorflow.keras.layers import Dense,Flatten, Conv2D
from tensorflow.keras import Model

Loading and preparing MNIST dataset

In [0]:
mnist = tf.keras.datasets.mnist
(x_train,y_train),(x_test,y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0


In [0]:
#Adding a channel dimension
x_train = x_train[..., tf.newaxis]
x_test = x_test[..., tf.newaxis]

Using tf.data to batch and shuffle the dataset:

In [0]:
train_ds = tf.data.Dataset.from_tensor_slices((x_train,y_train)).shuffle(10000).batch(32)
test_ds = tf.data.Dataset.from_tensor_slices((x_train,y_train)).batch(32)

Building the tf.keras model using the Keras model subclassing API:


In [0]:
class MyModel(Model):
  def __init__(self):
    super(MyModel, self).__init__()
    self.conv1 = Conv2D(32,3,activation ='relu')
    self.flatten = Flatten()
    self.d1 = Dense(128, activation ='relu')
    self.d2 = Dense(10)

  def call(self, x):
    x = self.conv1(x)
    x = self.flatten(x)
    x = self.d1(x)
    return self.d2(x)

#creating an instance of the model
model = MyModel()

Choosing Optimiser and loss finction

In [0]:
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(from_logits = True)
optimizer = tf.keras.optimizers.Adam()

Selecting the metrics to measure the loss and the accuracy of the model. These metrics accumulate the values over epochs and then print the overall result

In [0]:
train_loss = tf.keras.metrics.Mean(name= 'train_loss')
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name = 'train_accuracy')

test_loss = tf.keras.metrics.Mean(name = 'test_loss')
test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name = 'test_accuracy')

Using tf.GradientTape to train the model:

In [0]:
@tf.function
def train_step(images,labels):
    with tf.GradientTape() as tape:
       # training=True is only needed if there are layers with different
      #behavior during training versus inference (e.g. Dropout).
      predictions = model(images, training= True)
      loss = loss_object(labels,predictions)
    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients,model.trainable_variables))

    train_loss(loss)
    train_accuracy(labels,predictions)


testing model

In [0]:
@tf.function
def test_step(images, labels):
   #training=False is only needed if there are layers with different
   #behavior during training versus inference (e.g. Dropout).
   predictions = model(images, training= True)
   t_loss = loss_object(labels,predictions)

   test_loss(t_loss)
   test_accuracy(labels,predictions)
   

In [0]:
EPOCHS = 5

for epoch in range(EPOCHS):
  # Reset the metrics at the start of the next epoch
  train_loss.reset_states()
  train_accuracy.reset_states()
  test_loss.reset_states()
  test_accuracy.reset_states()

  for images, labels in train_ds:
    train_step(images,labels)

  for test_images, test_labels in test_ds:
    test_step(test_images,test_labels)
  
  template = 'Epoch {}, Loss: {}, Accuracy:{}, Test Loss: {}, Test Accuracy: {}'
  print(template.format(epoch + 1,
                        train_loss.result(),
                        train_accuracy.result() *100,
                        test_loss.result(),
                        test_accuracy.result() *100))

               



If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

Epoch 1, Loss: 0.13493454456329346, Accuracy:95.93499755859375, Test Loss: 0.04387829452753067, Test Accuracy: 98.77832794189453
Epoch 2, Loss: 0.04148292541503906, Accuracy:98.6816635131836, Test Loss: 0.02407815307378769, Test Accuracy: 99.23500061035156
Epoch 3, Loss: 0.022648649290204048, Accuracy:99.29999542236328, Test Loss: 0.014492865651845932, Test Accuracy: 99.56832885742188
Epoch 4, Loss: 0.01391378603875637, Accuracy:99.5566635131836, Test Loss: 0.006173391826450825, Test Accuracy: 99.819999

The image classifier is now trained to 99% accuracy on this dataset. 